In [135]:
from nbconvert.preprocessors import ExecutePreprocessor
ExecutePreprocessor.timeout=3000

In [136]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from string import punctuation
from nltk.tokenize import RegexpTokenizer
import re
import collections
from past.builtins import xrange
import spacy
nlp=spacy.load('en_core_web_sm')

In [137]:
#pointing the source and listing the file names under that path
path = os.chdir('C://Users//ar393556//Documents//Ticket grams//sample')
files = os.listdir(path)
files

['back 1-ticket_df_with_topgrams_automation_category.csv',
 'bigrams.csv',
 'data_50k.xlsx',
 'dict_file.csv',
 'Diff_words.csv',
 'final words.csv',
 'fourgram.csv',
 'sample_words.txtdic',
 'ticket_df_with_bigram.csv',
 'ticket_df_with_topgrams.csv',
 'trigram.csv',
 'uigrams.csv']

In [138]:
#Fetching only xlsx files
files_xls = [f for f in files if f[-4:] == 'xlsx']
files_xls

['data_50k.xlsx']

In [139]:
df = pd.DataFrame()

#Reading the files and appending the data in the DataFrame format
for f in files_xls:
    data = pd.read_excel(f,sheet_name='CI-Data')
    df = df.append(data)

In [140]:
df.shape

(49994, 36)

In [141]:
#Taking only TicketID and Summary which are essential

ticket_df_full = pd.DataFrame(df, columns= ['TicketID','Summary'])

ticket_df_trim=ticket_df_full[['TicketID','Summary']]

#no. of records and columns
ticket_df_trim.shape

#Dropping null/blank records
ticket_df_trim=ticket_df_trim.dropna()

In [142]:
ticket_df_full = ticket_df_trim[ticket_df_trim['Summary'].notnull()]

In [143]:
#New table data shape
ticket_df_full.shape

(49104, 2)

In [144]:
ticket_df_full.head()

,TicketID,Summary
0,10000,Cannot access clients in the training environm...
1,10001,cannot access Coin online.
2,10002,cannot access Coin.
3,10004,"cannot access euhreka via desktop application,..."
4,10005,Cannot access file stored in iManage Desksite


In [145]:
ticket_df_full['unclean']=ticket_df_full['Summary']

In [146]:
ticket_df_full.head()

,TicketID,Summary,unclean
0,10000,Cannot access clients in the training environm...,Cannot access clients in the training environm...
1,10001,cannot access Coin online.,cannot access Coin online.
2,10002,cannot access Coin.,cannot access Coin.
3,10004,"cannot access euhreka via desktop application,...","cannot access euhreka via desktop application,..."
4,10005,Cannot access file stored in iManage Desksite,Cannot access file stored in iManage Desksite


In [147]:
ticket_df=ticket_df_full[48210:]

In [148]:
# To remove punchuation, numbers and converting the string to lower
ticket_df['Summary'] = ticket_df['Summary'].str.lower().str.replace(r'[^a-z\s]', '')


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [149]:
ticket_df.head()    

,TicketID,Summary,unclean
49100,4886,alarm buildartifactsdiskspaceutilizationampdgl...,"ALARM: ""/build/artifacts_DiskSpaceUtilization_..."
49101,4887,alarm datawpmdiskspaceutilizationampwpmcmsasp ...,"ALARM: ""/data/wpm_DiskSpaceUtilization_AMPWPMC..."
49102,49922,we need to reverse transaction as it is preve...,We need to reverse transaction 1360925904 as i...
49103,49923,we need to send some email to bpms uat pls tur...,We need to send some email to BPMS UAT. Pls tu...
49104,49925,we need to setup jb were datafeeds can you pro...,We need to setup JB Were datafeeds. Can you pr...


In [150]:
nlp.Defaults.stop_words.add('thanks')
nlp.Defaults.stop_words.add('please')
nlp.Defaults.stop_words.add('team')
nlp.Defaults.stop_words.add('dear')
nlp.Defaults.stop_words.add('hi')

In [151]:
nlp.Defaults.stop_words.remove('not')
nlp.Defaults.stop_words.remove('cannot')
nlp.Defaults.stop_words.remove('nothing')

KeyError: 'not'

In [152]:
stop_words=nlp.Defaults.stop_words

In [153]:
ticket_df.isnull().sum()

TicketID    0
Summary     0
unclean     0
dtype: int64

In [154]:
ticket_df=ticket_df.dropna()
ticket_df.isnull().sum()

TicketID    0
Summary     0
unclean     0
dtype: int64

In [155]:


ticket_df['Summary'] = ticket_df['Summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))

In [156]:
ticket_df['Summary'] = ticket_df['Summary'].apply(lambda x: " ".join(x for x in x.split() if len(x)>=2 and len(x)<15))

In [157]:
%%time
ticket_df['Summary']= ticket_df['Summary'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

Wall time: 12.5 s


In [158]:
ticket_df.head()

,TicketID,Summary,unclean
49100,4886,alarm asia pacific sydney,"ALARM: ""/build/artifacts_DiskSpaceUtilization_..."
49101,4887,alarm asia pacific sydney,"ALARM: ""/data/wpm_DiskSpaceUtilization_AMPWPMC..."
49102,49922,need reverse transaction prevent sell settle r...,We need to reverse transaction 1360925904 as i...
49103,49923,need send email bpm uat pls turn -PRON- would ...,We need to send some email to BPMS UAT. Pls tu...
49104,49925,need setup jb datafeed provide assistance,We need to setup JB Were datafeeds. Can you pr...


In [159]:


def get_ngrams(text, n ):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

In [160]:
# ticket_df.to_csv('final words.csv')

In [161]:

ui=collections.Counter()

for i in ticket_df['Summary']:
    x = i.rstrip().split(" ")
    ui.update(ngrams(x, n=1))

In [162]:
ui

Counter({('meeting',): 37,
         ('slowness',): 1,
         ('extra',): 1,
         ('amppsdmd',): 2,
         ('cgsmlt',): 1,
         ('map',): 1,
         ('design',): 9,
         ('remuneration',): 2,
         ('work',): 160,
         ('jennifer',): 1,
         ('directly',): 2,
         ('repository',): 1,
         ('status',): 6,
         ('boban',): 1,
         ('scv',): 1,
         ('ok',): 1,
         ('program',): 1,
         ('updated',): 1,
         ('welcome',): 1,
         ('lauren',): 2,
         ('sure',): 6,
         ('doesburg',): 1,
         ('invitation',): 1,
         ('nsw',): 4,
         ('take',): 4,
         ('suggestion',): 1,
         ('kindly',): 1,
         ('planner',): 6,
         ('timer',): 1,
         ('gpupdate',): 1,
         ('remainder',): 2,
         ('ilane',): 1,
         ('remain',): 2,
         ('certain',): 2,
         ('camera',): 16,
         ('norco',): 1,
         ('page',): 12,
         ('know',): 13,
         ('ntp',): 1,
         ('

In [163]:
ui_counter = pd.DataFrame.from_dict(ui, orient='index')
ui_counter.to_csv('uigrams.csv')
ui_df=pd.read_csv('uigrams.csv',index_col=False)
ui_df['Unnamed: 0'] = ui_df['Unnamed: 0'].str.lower().str.replace(r'[^a-z\s]', '')
dict_ui=ui_df.set_index('Unnamed: 0')['0'].to_dict()

In [164]:
dict_ui

{'': 3,
 'delivery': 1,
 'hardware': 25,
 'consent': 1,
 'print': 5,
 'statement': 11,
 'content': 2,
 'dmld': 1,
 'effective': 1,
 'ne': 3,
 'suppose': 1,
 'key': 3,
 'disappear': 1,
 'cisci': 1,
 'mailbox': 3,
 'strategic': 1,
 'presentation': 2,
 'attend': 2,
 'kate': 1,
 'letter': 1,
 'uat': 1,
 'selfview': 11,
 'newupdat': 1,
 'ampconnect': 1,
 'tpd': 1,
 'rd': 2,
 'screenshot': 8,
 'ioo': 1,
 'strategy': 1,
 'amnpl': 1,
 'shankar': 2,
 'missing': 1,
 'lex': 1,
 'separately': 1,
 'stamatia': 1,
 'capital': 3,
 'issac': 1,
 'ampcrpshraspt': 1,
 'joint': 3,
 'pay': 1,
 'instal': 3,
 'nj': 3,
 'mitton': 1,
 'early': 2,
 'naidu': 1,
 'commsec': 1,
 'munro': 1,
 'handset': 2,
 'robert': 1,
 'compare': 1,
 'benjamin': 1,
 'northusm': 2,
 'stock': 2,
 'bluedoor': 1,
 'genusm': 1,
 'stent': 2,
 'loan': 2,
 'component': 4,
 'setting': 2,
 'job': 8,
 'dxclb': 1,
 'design': 9,
 'response': 5,
 'question': 3,
 'table': 4,
 'aka': 1,
 'choose': 1,
 'datum': 8,
 'bronze': 1,
 'info': 4,
 'unloc

In [165]:

bi=collections.Counter()
for i in ticket_df['Summary']:
    x = i.rstrip().split(" ")
    bi.update(set(zip(x[:-1],x[1:])))

In [166]:
bigram_counter = pd.DataFrame.from_dict(bi, orient='index')
bigram_counter.to_csv('bigrams.csv')
bigram_df=pd.read_csv('bigrams.csv',index_col=False)
bigram_df['Unnamed: 0'] = bigram_df['Unnamed: 0'].str.lower().str.replace(r'[^a-z\s]', '')
dict_bigram=bigram_df.set_index('Unnamed: 0')['0'].to_dict()

In [167]:

tri=collections.Counter()
for i in ticket_df['Summary']:
    x = i.rstrip().split(" ")
    tri.update(set(zip(x[:-2],x[1:-1],x[2:])))

In [168]:
trigram_counter = pd.DataFrame.from_dict(tri, orient='index')
trigram_counter.to_csv('trigram.csv')

In [169]:
trigram_df=pd.read_csv('trigram.csv',index_col=False)
trigram_df['Unnamed: 0'] = trigram_df['Unnamed: 0'].str.lower().str.replace(r'[^a-z\s]', '')
dict_trigram=trigram_df.set_index('Unnamed: 0')['0'].to_dict()

In [170]:

four=collections.Counter()

for i in ticket_df['Summary']:
    x = i.rstrip().split(" ")
    four.update(ngrams(x, n=4))

In [171]:
fourgram_counter = pd.DataFrame.from_dict(four, orient='index')
fourgram_counter.to_csv('fourgram.csv')
fourgram_df=pd.read_csv('fourgram.csv',index_col=False)
fourgram_df['Unnamed: 0'] = fourgram_df['Unnamed: 0'].str.lower().str.replace(r'[^a-z\s]', '')
dict_four=fourgram_df.set_index('Unnamed: 0')['0'].to_dict()

In [172]:

def top_grams(dicti, N):
    res_topgram=[]
    for text in ticket_df['Summary']:  

        comp_list=get_ngrams(str(text),N)
        n = {k: dicti[k] for k in comp_list if k in dicti}
        flag=bool(n)
        res=''
        if flag==True:
            res=max(n, key=n.get)
        else:
            res='No Pattern'
        res_topgram.append(str(res))
        
    return res_topgram

In [175]:
ticket_df.head()

,TicketID,Summary,unclean
49100,4886,alarm asia pacific sydney,"ALARM: ""/build/artifacts_DiskSpaceUtilization_..."
49101,4887,alarm asia pacific sydney,"ALARM: ""/data/wpm_DiskSpaceUtilization_AMPWPMC..."
49102,49922,need reverse transaction prevent sell settle r...,We need to reverse transaction 1360925904 as i...
49103,49923,need send email bpm uat pls turn -PRON- would ...,We need to send some email to BPMS UAT. Pls tu...
49104,49925,need setup jb datafeed provide assistance,We need to setup JB Were datafeeds. Can you pr...


In [176]:

res_unigram=top_grams(dict_ui,1)
ticket_df['Top1Grams']=res_unigram

In [177]:
ticket_df.head()

,TicketID,Summary,unclean,Top1Grams
49100,4886,alarm asia pacific sydney,"ALARM: ""/build/artifacts_DiskSpaceUtilization_...",sydney
49101,4887,alarm asia pacific sydney,"ALARM: ""/data/wpm_DiskSpaceUtilization_AMPWPMC...",sydney
49102,49922,need reverse transaction prevent sell settle r...,We need to reverse transaction 1360925904 as i...,message
49103,49923,need send email bpm uat pls turn -PRON- would ...,We need to send some email to BPMS UAT. Pls tu...,would
49104,49925,need setup jb datafeed provide assistance,We need to setup JB Were datafeeds. Can you pr...,need


In [132]:
df_samp['Top1Gram'] = pd.DataFrame(res_unigram) 
df_samp.head()

NameError: name 'df_samp' is not defined

In [76]:
len(res_trigram)

894

In [71]:

res_bigram=top_grams(dict_bigram,2)
ticket_df['Top2Grams']=pd.DataFrame({'Top2Grams':res_bigram})

len(res_bigram)

894

In [72]:
ticket_df.head()


,TicketID,Summary,unclean,Top1Grams,Top2Grams,Top3Grams
49100,4886,alarm asia pacific sydney,"ALARM: ""/build/artifacts_DiskSpaceUtilization_...",NaN,NaN,NaN
49101,4887,alarm asia pacific sydney,"ALARM: ""/data/wpm_DiskSpaceUtilization_AMPWPMC...",NaN,NaN,NaN
49102,49922,need reverse transaction prevent sell settle r...,We need to reverse transaction 1360925904 as i...,NaN,NaN,NaN
49103,49923,need send email bpm uat pls turn -PRON- would ...,We need to send some email to BPMS UAT. Pls tu...,NaN,NaN,NaN
49104,49925,need setup jb datafeed provide assistance,We need to setup JB Were datafeeds. Can you pr...,NaN,NaN,NaN


In [73]:
res_trigram=top_grams(dict_trigram, 3)
ticket_df['Top3Grams']=pd.DataFrame({'Top3Grams':res_trigram})

In [63]:
ticket_df.head()


,TicketID,Summary,unclean,Top1Grams,Top2Grams,Top3Grams
49100,4886,alarm asia pacific sydney,"ALARM: ""/build/artifacts_DiskSpaceUtilization_...",NaN,NaN,NaN
49101,4887,alarm asia pacific sydney,"ALARM: ""/data/wpm_DiskSpaceUtilization_AMPWPMC...",NaN,NaN,NaN
49102,49922,need reverse transaction prevent sell settle r...,We need to reverse transaction 1360925904 as i...,NaN,NaN,NaN
49103,49923,need send email bpm uat pls turn -PRON- would ...,We need to send some email to BPMS UAT. Pls tu...,NaN,NaN,NaN
49104,49925,need setup jb datafeed provide assistance,We need to setup JB Were datafeeds. Can you pr...,NaN,NaN,NaN


In [45]:

res_fourgram=top_grams(dict_four,4)
ticket_df['Top4Grams']=pd.DataFrame({'Top4Grams':res_fourgram})

In [46]:
ticket_df.tail()

,TicketID,Summary,unclean,Top1Grams,Top2Grams,Top3Grams,Top4Grams
49989,50300,user try register ampconz website receive atta...,When users are trying to register on amp.co.nz...,NaN,NaN,NaN,NaN
49990,6251,assist customer follow outstanding incident,Assisted customer and followed up on outstandi...,NaN,NaN,NaN,NaN
49991,5312,amp report th jan,AMP Report 04th Jan 2018,NaN,NaN,NaN,NaN
49992,5048,file drive delete,All files on my h: drive (\\ampnswfilerp05\bnh...,NaN,NaN,NaN,NaN
49993,5311,amp report feb,AMP Report 01 Feb 2019,NaN,NaN,NaN,NaN


In [47]:
ticket_df.to_csv('ticket_df_with_topgrams.csv')

In [48]:
path = os.chdir('C://Users//ar393556//Documents//Ticket grams//CIKeyword')
files = os.listdir(path)

automation_df=pd.read_excel('CIKeywords.xlsx')

In [49]:
automation_df.head()

,Keyword,Description,Group-Classification,AutomationCategory,Weight,SearchType
0,*overrunning*,NaN,NaN,Evaluate for Automation,1,0
1,*monitoring*,NaN,NaN,Service Improvement,1,0
2,*conference*,NaN,NaN,Service Improvement,1,0
3,*dutymanager*,NaN,NaN,Not Likely Automatable,1,0
4,*smartcard*,NaN,NaN,Evaluate for Automation,1,0


In [50]:
automation_df['Keyword']=automation_df['Keyword'].apply(lambda x: x.strip(punctuation))
automation_df['Keyword']=automation_df['Keyword'].str.replace('*',' ')

In [51]:
# To remove punchuation, numbers and converting the string to lower
automation_df['Keyword']=automation_df['Keyword'].str.lower()

In [52]:
%%time
automation_df['Keyword']=automation_df['Keyword'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

Wall time: 8.01 s


In [53]:
automation_df.head()

,Keyword,Description,Group-Classification,AutomationCategory,Weight,SearchType
0,overrun,NaN,NaN,Evaluate for Automation,1,0
1,monitor,NaN,NaN,Service Improvement,1,0
2,conference,NaN,NaN,Service Improvement,1,0
3,dutymanager,NaN,NaN,Not Likely Automatable,1,0
4,smartcard,NaN,NaN,Evaluate for Automation,1,0


In [54]:
dict_keyword=automation_df.set_index('Keyword')['Weight'].to_dict()


In [55]:
dict_automation_text=automation_df.set_index('Keyword')['AutomationCategory'].to_dict()

In [56]:
len(ticket_df)
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [57]:
# grams_list_record=[]
# print(ticket_df['Top2Grams'][391])
# # grams_list_record.append(ticket_df['Top2Grams'][375])
ticket_df.head()

,TicketID,Summary,unclean,Top1Grams,Top2Grams,Top3Grams,Top4Grams
0,10000,can not access client training environment get...,Cannot access clients in the training environm...,not,can not,can not access,can not access client
1,10001,can not access coin online,cannot access Coin online.,not,can not,can not access,can not access coin
2,10002,can not access coin,cannot access Coin.,not,can not,can not access,can not access coin
3,10004,can not access euhreka desktop application lin...,"cannot access euhreka via desktop application,...",not,can not,can not access,can not access euhreka
4,10005,can not access file store imanage desksite,Cannot access file stored in iManage Desksite,not,can not,can not access,can not access file


In [58]:
# ticket_500=ticket_df[0:500]
# ticket_500.shape

In [59]:
# for index,row in ticket_500.iterrows():
#     print(ticket_500['Top2Grams'][index])

In [83]:
twoK_df=ticket_df[47000:]

In [84]:
twoK_df.shape

(2102, 8)

In [85]:
twoK_df.head()

,TicketID,Summary,unclean,Top1Grams,Top2Grams,Top3Grams,Top4Grams,Automation Category
47889,6842,assist customer follow outstanding incident,Assisted customer and followed up on outstandi...,error,error message,error message appear,message appear plan find,NaN
47890,6843,assist customer follow outstanding incident,Assisted customer and followed up on outstandi...,user,error message,error message appear,file upload api require,NaN
47891,7306,assist customer follow outstanding incident,Assisted customer and followed up on outstandi...,need,unable view,pan control invert,presentation row light need,NaN
47892,7307,assist customer follow outstanding incident,Assisted customer and followed up on outstandi...,not,do not,do not correct,position test camera control,NaN
47893,7308,assist customer follow outstanding incident,Assisted customer and followed up on outstandi...,leave,camera face,camera face leave,camera face leave tv,NaN


In [86]:
%%time
automation_col=[]
# for i in columns:
# for record in range(0,len(ticket_df)):
for index,row in twoK_df.iterrows():
    
#     for index, row in df.iterrows():
#     print(index)
    
    
    grams_list_record=[]
    grams_list_record.append(str(twoK_df['Top1Grams'][index]))
    grams_list_record.append(str(twoK_df['Top2Grams'][index]))
    grams_list_record.append(str(twoK_df['Top3Grams'][index]))
    li=[]
#     li=[i for grams_words in grams_list_record for i in dict_keyword if fuzz.token_sort_ratio(grams_words,i)>80]
    for grams_words in grams_list_record:
        
        highest = process.extractOne(grams_words,dict_keyword.keys()  , scorer=fuzz.token_set_ratio)
        #scorer=fuzz.QRatio ,  scorer=fuzz.ratio
        li.append(highest[0])
#         for dict_words in dict_keyword:
#             if fuzz.token_sort_ratio(grams_words,dict_words)>80:
#                 li.append(dict_words)
#                 break
    
    n = {k: dict_keyword[k] for k in li if k in dict_keyword}
#     print(grams_list_record)
#     print(n)
    print(index)
    flag=bool(n)
    res=''
    if flag==True:
        res=max(n, key=n.get)
    else:
        res=''
    #print(res)
    automation_col.append(res)

47889
47890
47891
47892
47893
47894
47895
47896
47897
47898
47899
47900
47901
47902
47903
47904
47905
47906
47907
47908
47909
47910
47911
47912
47913
47914
47915
47916
47917
47918
47919
47920
47921
47922
47923
47924
47925
47926
47927
47928
47929
47930
47931
47932
47933
47934
47935
47936
47937
47938
47939
47940
47941
47942
47943
47944
47945
47946
47947
47948
47949
47950
47951
47952
47953
47954
47955
47956
47957
47958
47959
47960
47961
47962
47963
47964
47965
47966
47967
47968
47969
47970
47971
47972
47973
47974
47975
47976
47977
47978
47979
47980
47981
47982
47983
47984
47985
47986
47987
47988
47989
47990
47991
47992
47993
47994
47995
47996
47997
47998
47999
48000
48001
48002
48003
48004
48005
48006
48007
48008
48009
48010
48011
48012
48013
48014
48015
48016
48017
48018
48019
48020
48021
48022
48023
48024
48025
48026
48027
48028
48029
48030
48031
48032
48033
48034
48035
48036
48037
48038
48039
48040
48041
48042
48043
48044
48045
48046
48047
48048
48049
48050
48051
48052
48053
48054
4805

In [61]:
ticket_df.iloc[48198]

TicketID                                                  9671
Summary      calm scenario pretax salary sacrifice mortgage...
unclean      Calm Scenario for pre-tax salary Sacrifice for...
Top1Grams                                                  amp
Top2Grams                                           amp report
Top3Grams                                        amp report rd
Top4Grams                                           No Pattern
Name: 49090, dtype: object

In [62]:
print(5)

5


In [63]:
ticket_df.to_csv('checking.csv')

In [63]:
ticket_df.iloc[376]

TicketID                                                 10452
Summary      case request -PRON- would user register manage...
unclean      Case Request ID: 1038722132  User: Registers M...
Top1Grams                                                  not
Top2Grams                                              can not
Top3Grams                                      can not proceed
Top4Grams                      case note milestone outstanding
Name: 377, dtype: object

In [64]:
len(automation_col)

48210

In [65]:
automation_col

['end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'new server',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'bad file',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'local admin',
 'check job',
 'access request user',
 'service broker',
 'end not',
 'end not',
 'end not',
 'end not',
 'can not login',
 'can not login',
 'can not login',
 'can not login',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',
 'end not',

In [66]:
automation_col[2555]

'not print'

In [67]:
len(dict_automation_text)

562

In [64]:
# final_values = {k: dict_automation_text[k] for k in automation_col if k in dict_automation_text }
automation_text = [dict_automation_text[k] for k in automation_col if k in dict_automation_text]
# automation_text=list(final_values.values())

In [65]:
ticket_df['Automation Category']=pd.DataFrame({'Automation Category':automation_text})


In [66]:
ticket_df.to_csv('ticket_df_with_topgrams_automation_category_48K.csv')

In [ ]:
import nltk
nltk.edit_distance("laptop log", "log laptop")

In [ ]:
# import spacy 
  
# nlp = spacy.load('en_core_web_md') 
  
# print("Enter two space-separated words") 

# tokens = nlp(words) 
  


In [ ]:

fuzz.token_sort_ratio('lopfail run', 'run laopfail')

In [ ]:
# temp_list=['sensor', 'review doc', 'database off', 'over run', 'standby database', 'tradelink access', 'cpu high', 'server busy', 'outlook', 'check point', 'over run', 'standby database', 'tradelink access', 'cpu high', 'server busy', 'outlook', 'check point', 'over run', 'standby database', 'review doc', 'database off', 'over run', 'standby database', 'tradelink access', 'cpu high', 'server busy', 'outlook', 'check point', 'over run', 'standby database', 'tradelink access', 'cpu high', 'server busy', 'outlook', 'check point', 'over run', 'standby database', 'review doc', 'database off', 'over run', 'standby database', 'tradelink access', 'cpu high', 'server busy', 'outlook', 'check point', 'over run', 'standby database', 'tradelink access', 'cpu high', 'server busy', 'outlook', 'check point', 'over run', 'standby database', 'review doc', 'database off', 'over run', 'standby database', 'tradelink access', 'cpu high', 'server busy', 'outlook', 'check point', 'over run', 'standby database', 'tradelink access', 'cpu high', 'server busy', 'outlook', 'check point', 'over run', 'standby database', 'tradelink access', 'cpu high', 'server busy', 'outlook', 'check point', 'latency', 'networker backup', 'loan laptop', 'archive', 'access transaction', '996 byte', 'drive full', '', 'disk', 'jam', 'cluster fail', 'cti checkout', 'not print', 'oracle patch', 'oracle pwd reset', 'bad file', 'database avail', 'disk high', 'battery', 'freq lock', 'reset pass oracle', 'overrun', 'network access', 'home drive', 'master datum', 'decomm vol', 'write limit', 'snmp', 'auto - rerun', 'resolve', 'below incident', 'shut', 'interface fail', 'access sap', 'service miss', 'database avail cursor']
# temp_list=['laptop','high cpu' , 'bandwidth']
temp_list=['lopfail run']

In [ ]:
list_temp_comp=['run laopfail']

In [ ]:
%%time
li=[i for grams_words in temp_list for i in list_temp_comp if fuzz.token_sort_ratio(grams_words,i)>80]
print(li)

In [ ]:
%%time
li=[]
for grams_words in temp_list:
    for i in dict_keyword:
        if fuzz.token_sort_ratio(grams_words,i)>80:
            li.append(i)

# n = {k: dict_keyword[k] for k in li if k in dict_keyword}
# print(n)
# res=''
# if flag==True:
#     res=max(n, key=n.get)
# else:
#     res=''
# print(res)

In [ ]:
ci_keywords_list=[]

In [ ]:
dict_keyword.keys()

In [ ]:
strOptions=['', 'snmp', 'home drive', 'port', 'active install', 'ad site', 'extension number', 'database avail block', 'guide', 'below incident', 'sap wf access', 'cpu util', 'heuristic', 'disk high', 'apple', 'usb', 'query', 'can not login', 'project install', 'performance desktop', 'avaya', 'accou lock', 'clussvc', 'sim card', 'locus', 'active password', 'automate check', 'flash install', 'reset pwd sap', 'printer install', 'patrol down', 'excel install', 'trap storm', 'monitor upgrade', 'user unabl   print', 'pr4 access', 'process time', 'please process', 'breakglass', 'bill', 'shut', 'kernel patch', 'password lan', 'process count', 'pfa', 'upgrade', 'space log segment', 'power supply', 'restart', 'port down', 'connect', 'loan laptop', 'new server', 'db space', 'response time', 'false possitive', 'internet issue', 'account user lock', 'desktop upgrade', 'good', 'drive full', 'tablet', 'presentation', 'exception', 'server busy', 'space request', 'monitor', '  move', 'login', 'log comp', 'agent not run', 'reservation', '9 monitor', 'sanity', 'log lock', 'log network', 'performance compu', 'local admin', 'file system full', 'backup job fail', 'down server', 'mymail', 'outlook', 'account lock user', 'down', 'pds miss', 'long run', 'delete user', 'process stop', 'remove lun', 'ad reset', 'sap reset pwd', 'password sap', 'auto - rerun', 'hfm access request', 'collector', 'network decomm', 'app checkout', 'database space', 'share access', 'install', 'filesystem thre', 'drive space', 'hdd', 'monitor issue', 'standby database', 'bitlocker', 'miss process', 'exceed', 'disk thre', 'cpu free', 'proctime less', 'space free', 'manifest   failure', 'printer jam', 'enquiry', 'office install', 'hung compu', 'monitor config', 'service terminate', 'bad file', 'atp issue', 'write limit', 'sla miss', 'cluster', 'monitor problem', 'job backup fail', 'sign desk', 'adobe install', 'alert down', 'shutdown', 'hold space', 'sale order', 'window phone', 'sybase', 'computer down', 'failover', 'outlook inst', 'end not', 'monitor deploy', 'have fail', 'version main', 'socket', 'request pr4 access', 'authorization', 'client install', 'myaccess', 'user account lock', 'map printer', 'ora error', 'inquiry', 'morning', 'block process', 'how to', 'tradelink access', 'datastore', 'add tb', 'lan cable', 'connect server', 'link down', 'user lock account', 'allocate', 'database avail archive', 'alert cpu utili', 'lock lan', 'ping fail', 'slow desktop', 'cpu state', 'performance laptop', 'batch job', 'not connect', 'provision', 'database avail check log', 'mount', 'service run', 'memor high', 'interface fail', 'unable login', 'service miss', 'deadlock', 'sim activ', 'service not run', 'service bus', 'power install', 'block queue', 'domain service', 'verita', 'dutymanager', 'file witness', 'log space', 'unable conect', 'node down', 'tablespace', 'ping', 'time out reading', 'samsung', 'eod check', 'physical drive', 'slow compu', 'cold start', 'process run', 'space database', 'freeze compu', 'good phone', 'filesystem free', 'replace monitor', 'cti checkout', 'cpu thre', 'boot', '  health', 'restart service', 'access sap', 'cpu critical', 'latency', 'memory pool', 'wait time', 'sign syste', 'move database', 'power', 'windo password', 'upgrade ram', 'run too long', 'bandwidth', 'print install', 'client monitor', 'pwd reset oracle', 'qdepthinput', 'block', 'citrix license', 'toner', 'fail connect', 'install sap', 'network access', 'good pin', 'add server', 'assistance', 'patch install', 'disk abort', 'note monitor', 'utili cpu', 'access pr4', 'cpux', 'idoc issue', 'lync headset', 'sign comp', 'swap space', 'log wind', 'file miss', 'keyboard', 'degrade', 'iphone', 'direct install', 'backup fail', 'payment', 'sod check', 'queue', 'ping response', 'kernel latency', 'need access', 'user monitor', 'sw install', 'insert', 'agent reach', 'cpu high', 'desktop', 'ad lock', 'database unavail', 'freeze laptop', 'map folder', 'check point', 'backup failure', 'share drive', 'add gb', 'check kps', 'sla violate', 'disk space', 'bill issue', 'laptop', 'can not log', 'excel', 'note callback', 'test', 'queue threshold', 'software install', 'freeze desktop', 'hourly check', 'firefox install', 'check status', 'sap reset pass', 'workstation', 'bill doc', 'page', 'wintel request', 'monitor display', 'memory critical', 'switch', 'database off', 'ping webdisp', 'hang desktop', 'sign network', 'print add', 'oracle patch', 'database avail', 'service broker', 'switch down', 'servletexception', 'cpu usage', 'pdf', 'print conf', 'job status', 'mydesk', 'map network', 'call back', 'battery', 'decomm vol', 'sign laptop', 'emailqueue', 'active lock', 'database avail ora - error', 'ipad', 'notok', 'access request', 'service', 'access request user', 'space thre', 'netbackup', 'archive', 'hang compu', 'screen', 'error log', 'log syste', 'not ready', 'high processor load', 'database avail cursor', 'userid be lock', 'oracle pwd reset', 'airwatch', 'wireless', 'cluster degrad', 'visio install', 'database avail dump', 'conference', 'monitor install', 'logical_disk', 'memor free', 'chrome install', 'microsoft install', 'disk capacity', 'internet slow', 'replication', 'ghost call', 'computer not access', 'cluster fail', 'delivery', 'sensor', 'exit', 'job fail', 'error process request', 'memory utili', 'disk util', 'can not logon', 'heartbeat', 'oracle reset pass', 'memory', 'migration', 'pc', 'filesystem', 'monitor flick', '  ie', 'service stop', 'backup', 'user delete', 'charger', 'review doc', 'offline', 'network', 'config printer', 'autodiscovery', 'need access sap', 'daily check', 'citrix fail', 'java install', 'sap pwd reset', 'mirror', 'blank call', 'mailbox', 'print config', 'request access sap', 'mouse', 'monitor working', 'hardware', 'good password', 'request patch main', 'parse', 'callback', 'monitor resol', 'process -PRON- would fail', 'slow pc', 'duty manager', 'filesystem util', 'reset pwd oracle', 'new hire', 'iconnect', 'datum decomm', 'new join', 'batchqueue', 'incident monitor', 'otp', 'lync', 'vm', 'batch job fail', 'sap pass reset', 'gateway unavail', 'package fail', 'asset update', 'hang laptop', 'process down', 'trap', 'communication error', 'qdeptherror', 'app install', 'pass reset sap', 'domain controller inconsist', 'user unable login', 'pwd reset sap', 'slow laptop', 'access transaction', 'job duration', 'cpu', 'plus', 'file monitor', 'server decomm', 'high pag', 'delivery note', 'restart fail', 'reset pass sap', 'check job', 'pricing error', 'datum down', 'reset pass oracle', 'networker backup', 'server issue', 'purchase order', 'decomm', 'lumia', 'access access', 'agent down', 'unix / linux request', 'access', 'workspace', 'printer', 'plan order', 'memor util', 'virtual machine', 'process', 'memory usage', 'access pr2 pr4', 'incompatible', 'abend', 'log laptop', 'batch issue', 'process limit', 'oracle reset pwd', 'firewall', 'smartcard', 'proctime', 'interface down', 'out space', 'sign wind', 'space util', 'smart card', 'master datum', '996 byte', 'heart', 'intranet issue', 'not print', 'patch', 'process miss', 'memor thre', 'describe incident', 'intranet slow', 'dock', 'invoice', 'space', 'password', 'idoc error', 'service down', 'spare disk', 'purchase requisition', 'monitor blank', 'lock account user', 'duplicate', 'reboot', 'oracle pass reset', 'citrix', 'over run', 'disk', 'access folder', 'userid lock', 'process not run', 'active reset', 'extension dial', 'o365 install', 'app decomm', 'last backup', 'leaver', 'acc lock', 'word install', 'access sap wf', 'mobile', 'windo lock', 'overrun', 'name search', 'jam', 'production status', 'blackberry', 'lock user account', 'unable access', 'reset password', 'agent monitor', 'dimm', 'paper jam', 'android', 'outlook config', 'space high', 'pass reset oracle', 'resolve', 'filesystem high', 'mail config', 'server fail', 'good issue', 'windo reset', 'drive access', 'drive map', 'disk free', 'ad password', 'bloomberg', 'post period', 'database connect', 'queue delay', 'headset', 'health check', 'reset', 'cpu utilization', 'domain miss', 'skype install', 'drive', 'hard disk', 'speaker', 'lan port', 'pr4 access add', 'vlan', 'lock user', 'certificate', 'remove mail', 'build', 'desktop setup', 'batch fail', 'log desk', 'storage request', 'map drive', 'freq lock']

In [ ]:
%%time
from fuzzywuzzy import process
str2Match = [" ORACLE PASS RESET","outlook config", "space high"," ORACLE PASS RESET","outlook config", "space high"]
#strOptions = ["shop apple inc","apple park","apple incorporated","iphone"]
for i in str2Match:
    #highest = process.extractOne(i,dict_keyword.keys())
    highest = process.extract(i,dict_keyword.keys(), scorer=fuzz.ratio)[:1]
    print(highest)

#     Ratios = process.extract(i,strOptions, score_cutoff=80, limit=5)
#     print(Ratios)

In [ ]:
highest[1]

In [ ]:
process.extractOne()

In [ ]:
from ngram import NGram
n = NGram(["ham","spam","eggs"])
sorted(n.items_sharing_ngrams("mam").items())


In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame([{'c1':10, 'c2':100}, {'c1':11,'c2':110}, {'c1':12,'c2':120}])

In [ ]:
df.head()

In [ ]:
for index, row in df.iterrows():
    print(ticket_df['Top1Gramss'][index])

In [ ]:
print(5)